In [6]:
# Step 1: Load the dataset

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from gensim.models import Word2Vec
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.manifold import TSNE

file_path = "/home/guilherme/Documents/Github/Tese/Documentation/Dataset_Augmentation/Raw_Augmented_database_13_10_2023.xlsx"
df = pd.read_excel(file_path, sheet_name="Augmented Data")

FileNotFoundError: [Errno 2] No such file or directory: '/home/guilherme/Documents/GitHub/Tese/Documentation/Dataset_Augmentation/Raw_Augmented_database_13_10_2023.xlsx'

In [ ]:
# Step 2: Prepare the data: Create a list of symptom sequences for each disease
symptom_cols = [f"Symptom_{i}" for i in range(1, 26)]
symptoms_data = df[symptom_cols].values.tolist()

# Filtering out NaN values from symptoms
symptoms_data = [list(filter(lambda v: v==v, lst)) for lst in symptoms_data]  # v==v is a trick to check for NaN


In [ ]:
# Step 3: Train Word2Vec embeddings on these sequences

embedding_size = 128
w2v_model = Word2Vec(sentences=symptoms_data, vector_size=embedding_size, window=5, min_count=1, workers=4)


In [ ]:
# Step 4: Exploratory data analysis (EDA)
print(df.describe())
print(df.info())

# %%
# EDA Step 2: Distribution of the target variable (ICD 11 codes)

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6))
top_n_diseases = 50  # You can change this as needed
sns.countplot(y=df["ICD 11"], order=df["ICD 11"].value_counts().head(top_n_diseases).index)
plt.title("Distribution of Top ICD 11 Codes")
plt.ylabel("ICD 11 Codes")
plt.xlabel("Number of Occurrences")
plt.show()

# %%
# EDA Step 3: Distribution of the number of symptoms per record

symptom_counts = df[symptom_cols].count(axis=1)
plt.figure(figsize=(10, 6))
sns.histplot(symptom_counts, kde=False, bins=30, color="skyblue")

# Determine where 80% of the data falls
sorted_symptom_counts = sorted(symptom_counts)
index_10_percent = int(0.10 * len(sorted_symptom_counts))
index_90_percent = int(0.90 * len(sorted_symptom_counts))
lower_bound = sorted_symptom_counts[index_10_percent]
upper_bound = sorted_symptom_counts[index_90_percent]

# Highlight the range where 80% of occurrences fall
plt.axvspan(lower_bound, upper_bound, color='red', alpha=0.3)
plt.title("Distribution of Number of Symptoms per Record")
plt.xlabel("Number of Symptoms")
plt.ylabel("Number of Records")
plt.show()

# %%
# EDA Step 4: Visualization of Word2Vec embeddings using dimensionality reduction (TSNE)

from sklearn.manifold import TSNE

top_n_symptoms = 40
top_symptoms = pd.Series([item for sublist in symptoms_data for item in sublist]).value_counts().head(top_n_symptoms).index.tolist()
top_symptoms_embeddings = np.array([w2v_model.wv[symptom] for symptom in top_symptoms])

tsne = TSNE(n_components=2, random_state=42, perplexity=30)  # Adjust perplexity based on data size
reduced_embeddings = tsne.fit_transform(top_symptoms_embeddings)

plt.figure(figsize=(15, 10))
scatter = sns.scatterplot(x=reduced_embeddings[:, 0], y=reduced_embeddings[:, 1], alpha=0.8, s=80, hue=top_symptoms, palette='viridis')
plt.title("Visualization of Word2Vec Embeddings using TSNE")
plt.xlabel("TSNE Dimension 1")
plt.ylabel("TSNE Dimension 2")
scatter.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize='small')
plt.tight_layout()
# plt.savefig("Word2Vec_Visualization.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Step 5: Convert the symptom sequences into embeddings

def symptoms_to_embedding(symptoms, model, embedding_size):
    embedding_matrix = np.zeros((len(symptoms), embedding_size))
    for i, symptom in enumerate(symptoms):
        if symptom in model.wv:
            embedding_matrix[i] = model.wv[str(symptom)]
    return embedding_matrix

X_embedded = np.array([symptoms_to_embedding(patient_symptoms, w2v_model, embedding_size) for patient_symptoms in symptoms_data])
X_padded = pad_sequences(X_embedded, padding='post')

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df["ICD 11"])

In [ ]:
# Step 6: Class balancing through oversampling

# Class Imbalance: If certain classes (diseases) have significantly more samples than others, 
# you might need to address this imbalance using techniques like oversampling, undersampling,
#  or using metrics that are sensitive to imbalances


# Addressing class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_padded.reshape(X_padded.shape[0], -1), y_encoded)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=42, stratify=y_resampled)


# Compute the class weights.
from sklearn.utils.class_weight import compute_class_weight

unique_classes = np.unique(y_encoded)
class_weights = compute_class_weight('balanced', classes=unique_classes, y=y_encoded)
class_weight_dict = dict(zip(unique_classes, class_weights))


from keras.utils import to_categorical

y_train_encoded = to_categorical(y_train, num_classes=len(unique_classes))
y_test_encoded = to_categorical(y_test, num_classes=len(unique_classes))

max_sequence_length = X_padded.shape[1]
X_train = X_train.reshape(-1, max_sequence_length, 128)
X_test = X_test.reshape(-1, max_sequence_length, 128)


In [ ]:
# Step 7: Creating the model architecture (LSTM with embeddings)

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras_tuner.tuners import RandomSearch

n_classes = len(np.unique(y_encoded))
print(n_classes)
# Define the model-building function for LSTM
def build_lstm_model(hp):
    model = Sequential()

    model.add(LSTM(units=hp.Int('lstm_units_1', min_value=64, max_value=256, step=64), 
                   input_shape=(X_padded.shape[1], w2v_model.vector_size), 
                   return_sequences=True))

    model.add(LSTM(units=hp.Int('lstm_units_2', min_value=64, max_value=256, step=64)))

    model.add(Dense(units=hp.Int('dense_units_1', min_value=256, max_value=1024, step=256), 
                    activation='relu'))

    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(units=hp.Int('dense_units_2', min_value=128, max_value=512, step=128), 
                    activation='relu'))

    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(n_classes, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Define the tuner
tuner = RandomSearch(
    build_lstm_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=2,
    directory='hyperparam_search',
    project_name='disease_prediction_lstm'
)

# Search for the best model
tuner.search(X_train, y_train_encoded, epochs=10, validation_split=0.2)

# Get the best hyperparameters and build the model
best_hyperparameters = tuner.get_best_hyperparameters()[0]
print(best_hyperparameters.values)
model = tuner.hypermodel.build(best_hyperparameters)

model.summary()




In [ ]:
## Step 5. Training the Model

history = model.fit(X_train, y_train_encoded, validation_data=(X_test, y_test_encoded), epochs=20, batch_size=64, class_weight=class_weight_dict)


In [ ]:
# Step 7: Model Evaluation

loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f"Test Loss: {loss}")
print(f"Accuracy of the model: {accuracy * 100:.2f}%")

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_classes))



In [ ]:
# Step 11: Visualize the training progress
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Step 8: Model Deployment
import joblib

# Save the model, label encoder, and binarizer for later use
model_filename = 'Raw_Embedded_model.pkl'
w2v_model_filename = 'raw_w2v_model.pkl'
X_padded_filename = 'raw_X_padded.pkl'
# binarizer_filename = 'no_features_symptoms_binarizer.pkl'

joblib.dump(model, model_filename)
joblib.dump(w2v_model, w2v_model_filename)
joblib.dump(X_padded, X_padded_filename)

loaded_model = joblib.load(model_filename)
w2v_model_encoder = joblib.load(w2v_model_filename)
loaded_binarizer = joblib.load(X_padded_filename)



In [ ]:
# Step 12: Making Predictions (Optional)

def predict_top_diseases_with_confidence(symptom_codes, top_n=5):
    # Convert input symptoms to embeddings
    embedded_symptoms = symptoms_to_embedding(symptom_codes, w2v_model_encoder, embedding_size)
    embedded_symptoms = np.expand_dims(embedded_symptoms, 0)  # Add batch dimension
    
    # Pad the embedded symptoms
    padded_symptoms = pad_sequences(embedded_symptoms, maxlen=loaded_binarizer.shape[1], padding='post')

    # Predict using the model
    prediction = loaded_model.predict(padded_symptoms)

    # Get top N class indices based on highest probabilities
    top_n_indices = np.argsort(prediction[0])[-top_n:][::-1]
    top_n_probabilities = prediction[0][top_n_indices]

    # Decode these indices to get the ICD 11 codes
    top_n_icd_codes = label_encoder.inverse_transform(top_n_indices)

    # Zip together the ICD codes and their respective probabilities
    results = list(zip(top_n_icd_codes, top_n_probabilities))

    return results

# Example usage:
symptom_input = ['Fatigue', 'Shoulder pain', 'Leg pain', 'Muscle pain', 'Hip pain', 'Irregular heartbeat', 'Stiffness all over', 'Temper problems', 'Morning stiffness', 'Limited range of motion', 'Anorexia'] # FA22	Polymyalgia rheumatica
predictions_with_confidence = predict_top_diseases_with_confidence(symptom_input)
for icd, confidence in predictions_with_confidence:
    print(f"Predicted disease (ICD 11 Code): {icd} with confidence: {confidence:.2f}%")
